## Soft Linked

In [1]:
from pathlib import Path
import glob
import os
import pandas as pd

### Dataload

In [12]:
!tree  -L 2 /home/chenyi/workspace/dataset/deepedia/

/home/chenyi/workspace/dataset/deepedia/
├── deepedia6
├── info
│   ├── csv
│   ├── deepedia
│   └── pickle
├── labels94
│   ├── train
│   ├── train.cache
│   ├── val
│   └── val.cache
├── labels94.tar
├── labels_catg8
│   ├── train
│   ├── train.cache
│   ├── val
│   └── val.cache
└── labels_catg8.tar

11 directories, 6 files


In [11]:
!mkdir /home/chenyi/workspace/dataset/deepedia/deepedia6

In [13]:
dpedia_clr = pd.read_csv('/home/chenyi/workspace/dataset/deepedia/info/deepedia/dpedia_clr_temp.csv')

In [15]:
dpedia_clr.head(2)

,img_sp,id_bbox,width,height,mode,label8,bx1,bx2,bx3,bx4,source
0,/home/chenyi/workspace/dataset/data/data_fashi...,2,681,1024,train,2,0.551395,0.898926,53.0,127.0,fashionpedia
1,/home/chenyi/workspace/dataset/data/data_fashi...,3,681,1024,train,2,0.477974,0.869629,43.0,39.0,fashionpedia


In [20]:
dpedia_clr.value_counts('label8').sort_index()

label8
0     43178
1      7121
2    214911
3    119410
4     41089
5     49134
dtype: int64

#### 数据预处理

In [17]:
dpedia_clr['label8'] = dpedia_clr['label8'].values-2

In [22]:
dpedia_clr['bx3'] = dpedia_clr['bx3'].values / dpedia_clr['width'].values
dpedia_clr['bx4'] = dpedia_clr['bx4'].values / dpedia_clr['height'].values

In [32]:
dpedia_clr['fname'] = dpedia_clr['img_sp'].apply(
    lambda x: Path(x).name)

In [39]:
dpedia_clr['label_sp'] = dpedia_clr.apply(
    lambda row:'/home/chenyi/workspace/dataset/deepedia/deepedia6/{0}/{1}'.format(row['mode'], row['fname']), axis=1)

In [50]:
dpedia_clr['label_sp'] =  dpedia_clr['label_sp'].apply(lambda x:x.replace('.jpg', '.txt'))

In [66]:
dpedia_clr.head(2).values

array([['/home/chenyi/workspace/dataset/data/data_fashionpedia/train/0002ec21ddb8477e98b2cbb87ea2e269.jpg',
        2, 681, 1024, 'train', 0, 0.5513950073421439, 0.89892578125,
        0.07782672540381791, 0.1240234375, 'fashionpedia',
        '0002ec21ddb8477e98b2cbb87ea2e269.jpg',
        '/home/chenyi/workspace/dataset/deepedia/deepedia6/train/0002ec21ddb8477e98b2cbb87ea2e269.txt'],
       ['/home/chenyi/workspace/dataset/data/data_fashionpedia/train/0002ec21ddb8477e98b2cbb87ea2e269.jpg',
        3, 681, 1024, 'train', 0, 0.4779735682819383, 0.86962890625,
        0.0631424375917768, 0.0380859375, 'fashionpedia',
        '0002ec21ddb8477e98b2cbb87ea2e269.jpg',
        '/home/chenyi/workspace/dataset/deepedia/deepedia6/train/0002ec21ddb8477e98b2cbb87ea2e269.txt']],
      dtype=object)

### 生成label txt

In [51]:
df = dpedia_clr

In [54]:
%%time
input_list = df.apply(lambda x: 
                        [x['label_sp'], 
                         '{0:.0f} {1:.6f} {2:.6f} {3:.6f} {4:.6f}\n'.format(
                             *x.loc[['label8', 'bx1','bx2', 'bx3', 'bx4',]])], 
                        axis=1)

CPU times: user 2min 4s, sys: 0 ns, total: 2min 4s
Wall time: 2min 10s


In [55]:
input_list = input_list.values.tolist()

In [56]:
input_list[0]

['/home/chenyi/workspace/dataset/deepedia/deepedia6/train/0002ec21ddb8477e98b2cbb87ea2e269.txt',
 '0 0.551395 0.898926 0.077827 0.124023\n']

In [61]:
def write_txt(input_list):
    label_path, text = input_list[0], input_list[1]
    with open(label_path, 'a+') as f:
        f.write(text)

In [62]:
%%time
from utils.data.transferData import run_task_multithread
run_task_multithread(input_list, write_txt, thread_num=10)

CPU times: user 19.6 s, sys: 17.3 s, total: 36.8 s
Wall time: 36.6 s


In [63]:
imgs_list = glob.glob('/home/chenyi/workspace/dataset/deepedia/deepedia6/*/*.txt')
len(imgs_list)

260783

In [64]:
imgs_list = glob.glob('/home/chenyi/workspace/dataset/data/deepedia/images/*/*.jpg')
len(imgs_list)

269664

### target path

In [77]:
!tree  -d -L 2 /home/chenyi/workspace/yolov5/datasets/deepedia6

/home/chenyi/workspace/yolov5/datasets/deepedia6
├── images
│   ├── train
│   └── val
└── labels
    ├── train
    └── val

6 directories


In [76]:
!mkdir -p /home/chenyi/workspace/yolov5/datasets/deepedia6/labels/train

In [67]:
df = dpedia_clr.drop_duplicates('img_sp')

In [79]:
df.head(2)

,img_sp,id_bbox,width,height,mode,label8,bx1,bx2,bx3,bx4,source,fname,label_sp
0,/home/chenyi/workspace/dataset/data/data_fashi...,2,681,1024,train,0,0.551395,0.898926,0.077827,0.124023,fashionpedia,0002ec21ddb8477e98b2cbb87ea2e269.jpg,/home/chenyi/workspace/dataset/deepedia/deeped...
2,/home/chenyi/workspace/dataset/data/data_fashi...,6,682,1024,val,0,0.565249,0.833008,0.095308,0.130859,fashionpedia,00048c3a2fb9c29340473c4cfc06424a.jpg,/home/chenyi/workspace/dataset/deepedia/deeped...


In [83]:
df.head(2).values

array([['/home/chenyi/workspace/dataset/data/data_fashionpedia/train/0002ec21ddb8477e98b2cbb87ea2e269.jpg',
        2, 681, 1024, 'train', 0, 0.5513950073421439, 0.89892578125,
        0.07782672540381791, 0.1240234375, 'fashionpedia',
        '0002ec21ddb8477e98b2cbb87ea2e269.jpg',
        '/home/chenyi/workspace/dataset/deepedia/deepedia6/train/0002ec21ddb8477e98b2cbb87ea2e269.txt',
        '/home/chenyi/workspace/yolov5/datasets/deepedia6/images/train/0002ec21ddb8477e98b2cbb87ea2e269.jpg',
        '/home/chenyi/workspace/yolov5/datasets/deepedia6/labels/train/0002ec21ddb8477e98b2cbb87ea2e269.txt'],
       ['/home/chenyi/workspace/dataset/data/data_fashionpedia/train/00048c3a2fb9c29340473c4cfc06424a.jpg',
        6, 682, 1024, 'val', 0, 0.5652492668621701, 0.8330078125,
        0.09530791788856305, 0.130859375, 'fashionpedia',
        '00048c3a2fb9c29340473c4cfc06424a.jpg',
        '/home/chenyi/workspace/dataset/deepedia/deepedia6/val/00048c3a2fb9c29340473c4cfc06424a.txt',
        '

In [81]:
df['img_tp']  = df.apply(lambda x:
                         '/home/chenyi/workspace/yolov5/datasets/deepedia6/images/{0}/{1}'.format(x['mode'],
                                                                                                  x['fname']), axis=1)

/tmp/ipykernel_3099/2910940719.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['img_tp']  = df.apply(lambda x:


In [82]:
df['label_tp']  = df.apply(lambda x:
                         '/home/chenyi/workspace/yolov5/datasets/deepedia6/labels/{0}/{1}.txt'.format(x['mode'],
                                                                                                  x['fname'][:-4]), axis=1)

/tmp/ipykernel_3099/998469552.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label_tp']  = df.apply(lambda x:


In [295]:
# dfk.to_csv('/nas/chenyi/benchmark/deepedia24_val/deepedia_val1000.csv', index=False)

In [266]:
dfk = pd.read_csv('/nas/chenyi/benchmark/deepedia24_val/deepedia_val1000.csv')

In [273]:
dfk['img_tpath'] = dfk['img_path'].apply(lambda x:
                                        x.replace('/home/chenyi/workspace/yolov5/datasets/deepedia_catg24/',
                                                  '/nas/chenyi/benchmark/deepedia24_val/deepedia24_val1000/'))

### 软连接

In [84]:
!tree  -d -L 2 /home/chenyi/workspace/yolov5/datasets/deepedia6

/home/chenyi/workspace/yolov5/datasets/deepedia6
├── images
│   ├── train
│   └── val
└── labels
    ├── train
    └── val

6 directories


In [85]:
%%time
_ = df.apply(lambda row: os.symlink(row['img_sp'], row['img_tp']), axis=1)

CPU times: user 3.92 s, sys: 5.22 s, total: 9.14 s
Wall time: 11 s


In [86]:
%%time
_ = df.apply(lambda row: os.symlink(row['label_sp'], row['label_tp']), axis=1)

CPU times: user 3.46 s, sys: 6.15 s, total: 9.61 s
Wall time: 11.5 s


In [87]:
lsp = df.loc[0,'label_sp']
!cat $lsp

0 0.477974 0.869629 0.063142 0.038086
0 0.551395 0.898926 0.077827 0.124023
5 0.505874 0.447266 0.251101 0.416016


In [88]:
l = glob.glob('/home/chenyi/workspace/yolov5/datasets/deepedia6/images/*/*.jpg')
len(l)

260783

In [90]:
l = glob.glob('/home/chenyi/workspace/yolov5/datasets/deepedia6/labels/*/*.txt')
len(l)

260783